In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [2]:
import sklearn
from sklearn.model_selection import train_test_split

import numpy as np
import time
import matplotlib.pyplot as plt
import os

from glob import glob

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체
print(device)

#print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))

fontpath = 'C:/Windows/Fonts/NanumGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10).get_name()
plt.rc('font', family=font)

cpu


In [4]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

In [5]:
data_dir='C:/Users/kangb/capstoneDesign/Back-end/IdeaProjects/imageDownloader/Image'

In [6]:
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'style3'), transforms_train)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=0)

In [7]:
print('학습 데이터셋 크기:', len(train_datasets))
#print('테스트 데이터셋 크기:', len(valid_datasets))

class_names = train_datasets.classes
print('학습 클래스:', class_names)

def imshow(input, title):
    # torch.Tensor를 numpy 객체로 변환
    input = input.numpy().transpose((1, 2, 0))
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()

학습 데이터셋 크기: 582
학습 클래스: ['모던', '스포티', '캐주얼', '페미닌']


In [8]:
# 모델 경로
model_path = './weight/style_only_model_best_epoch.pth'

# 모델 로드
loaded_model = torch.load(model_path)

In [11]:
model = loaded_model
num_features = model.fc.in_features
# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 3개로 교체하여 마지막 레이어 다시 학습
# transfer learning
model.fc = nn.Sequential(     
    nn.Linear(num_features, 256),        # 마지막 완전히 연결된 계층에 대한 입력은 선형 계층, 256개의 출력값을 가짐
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_features),      # Since 10 possible outputs = 10 classes
    nn.LogSoftmax(dim=1)              # For using NLLLoss()
)


AttributeError: 'Sequential' object has no attribute 'in_features'

In [11]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

model = model.to(device)

num_epochs = 20

best_epoch = None
best_loss = 5

In [12]:
''' Train '''
# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    model.train()
    start_time = time.time()
    
    running_loss = 0.
    running_corrects = 0

    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_epoch = epoch
        print("best_loss: {:.4f} \t best_epoch: {}".format(best_loss, best_epoch))


#0 Loss: 2.3751 Acc: 29.0859% Time: 296.7641s
best_loss: 2.3751 	 best_epoch: 0
#1 Loss: 1.1644 Acc: 49.8615% Time: 353.6636s
best_loss: 1.1644 	 best_epoch: 1
#2 Loss: 0.8538 Acc: 64.2659% Time: 320.8868s
best_loss: 0.8538 	 best_epoch: 2
#3 Loss: 0.7297 Acc: 72.2992% Time: 278.8234s
best_loss: 0.7297 	 best_epoch: 3
#4 Loss: 0.7549 Acc: 74.2382% Time: 264.1837s
#5 Loss: 0.5646 Acc: 79.2244% Time: 258.0956s
best_loss: 0.5646 	 best_epoch: 5
#6 Loss: 0.5183 Acc: 80.3324% Time: 253.2672s
best_loss: 0.5183 	 best_epoch: 6
#7 Loss: 0.5335 Acc: 83.3795% Time: 237.2040s
#8 Loss: 0.4147 Acc: 86.7036% Time: 236.8358s
best_loss: 0.4147 	 best_epoch: 8
#9 Loss: 0.3893 Acc: 85.3186% Time: 235.0952s
best_loss: 0.3893 	 best_epoch: 9
#10 Loss: 0.4069 Acc: 88.0886% Time: 236.8759s
#11 Loss: 0.3502 Acc: 88.3656% Time: 236.2947s
best_loss: 0.3502 	 best_epoch: 11
#12 Loss: 0.3031 Acc: 89.7507% Time: 238.9359s
best_loss: 0.3031 	 best_epoch: 12
#13 Loss: 0.2067 Acc: 91.6898% Time: 237.0726s
best_loss:

In [13]:
os.makedirs('./weight',exist_ok=True)
torch.save(model,'./weight/style_only_model_best_epoch.pth')
